Progress So far:

We gathered some useful data sets from:

GenAge http://genomics.senescence.info/genes/

Cell Map(Yeast Genome): https://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/

Cell Map.org: https://thecellmap.org/

Data set : http://math.bu.edu/people/kolaczyk/datasets.html

Used the orthologs model to find similar genes between yeast, mice, worms, fruit flies

** TO DO **

* Need to look up Managbanag et al. (2008) "Shortest-path ..." study

* Need to formalize patterns for longevity genes/types

* Need to find protein interaction network for worm/mouse/or human

* mouse data connected via gene symbol

In [8]:
from copy import deepcopy
import json
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib
import matplotlib.pylab as plt
%matplotlib inline

**Load initial files**

- The 'yeast_genage_models_export.tsv' file contains the the genes which have been linked to longevity.

- The 'genage_models_orthologs_export.tsv' file contains orthologs from other species. 

Using .info() and .head() to take a look at type of data and formatting.

In [9]:
yeast_file = 'yeast_genage_models_export.tsv'
homologous_files = ['genage_models_orthologs_export.tsv']#, 'worm_genage_models_orthologs_export.tsv',
                    #'fly_genage_models_orthologs_export.tsv', 'mouse_genage_models_orthologs_export.tsv']
df_yeast = pd.read_csv(yeast_file, sep=r'\t', engine='python')

In [10]:
df_yeast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185 entries, 0 to 1184
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gene ID                  1185 non-null   int64  
 1   Entrez Gene ID           1185 non-null   int64  
 2   Gene Symbol              1185 non-null   object 
 3   Gene Name                1185 non-null   object 
 4   Organism                 1185 non-null   object 
 5   Unigene ID               0 non-null      float64
 6   Ensembl ID               1086 non-null   object 
 7   UniProt ID               0 non-null      float64
 8   Alias                    502 non-null    object 
 9   Lifespan Effect          1177 non-null   object 
 10  Phenotype Description    1185 non-null   object 
 11  Longevity Influence      1172 non-null   object 
 12  Max Lifespan Change      23 non-null     float64
 13  Avg Lifespan Change      634 non-null    float64
 14  Method                  

In [11]:
df_yeast.head(10)

Gene ID  Entrez Gene ID Gene Symbol                   Gene Name  \
0     1553          852380        AAC3             ADP/ATP Carrier   
1     1393          855581        AAH1      Adenine AminoHydrolase   
2     1612          853755        AAT1  Aspartate AminoTransferase   
3     1613          850714        AAT2  Aspartate AminoTransferase   
4     1683          850450        ABP1       Actin Binding Protein   
5     1683          850450        ABP1       Actin Binding Protein   
6     1683          850450        ABP1       Actin Binding Protein   
7     1683          850450        ABP1       Actin Binding Protein   
8     1507          852925        ACB1            Acyl-CoA-Binding   
9     1914          852266        ACH1                       Ach1p   

                   Organism  Unigene ID Ensembl ID  UniProt ID Alias  \
0  Saccharomyces cerevisiae         NaN    YBR085W         NaN  ANC3   
1  Saccharomyces cerevisiae         NaN    YNL141W         NaN   NaN   
2  Saccharomyces cerevisiae         NaN    YKL106W         NaN   NaN   
3  Saccharomyces cerevisiae         NaN    YLR027C         NaN  ASP5   
4  Saccharomyces cerevisiae         NaN    YCR088W         NaN   NaN   
5  Saccharomyces cerevisiae         NaN    YCR088W         NaN   NaN   
6  Saccharomyces cerevisiae         NaN    YCR088W         NaN   NaN   
7  Saccharomyces cerevisiae         NaN    YCR088W         NaN   NaN   
8  Saccharomyces cerevisiae         NaN    YGR037C         NaN   NaN   
9  Saccharomyces cerevisiae         NaN        NaN         NaN   NaN   

  Lifespan Effect                              Phenotype Description  \
0        increase  Replicative lifespan increased by 15% in the a...   
1        increase  Mean chronological lifespan increased in the a...   
2        increase              Replicative lifespan increased by 25%   
3        increase              Replicative lifespan increased by 15%   
4        increase  Replicative lifespan increased by 20% in the a...   
5        decrease  Replicative lifespan decreased by 20% in the a...   
6        increase  Replicative lifespan increased by 30% in the a...   
7        decrease  Replicative lifespan decreased by 20% in the a...   
8        increase  Mean replicative lifespan increased in the alp...   
9        decrease  Deletion of ACH1 reduces chronological lifespan.    

  Longevity Influence  Max Lifespan Change  Avg Lifespan Change  \
0                anti                  NaN                 15.0   
1                anti                  NaN                  NaN   
2                 pro                  NaN                 25.0   
3                 pro                  NaN                 15.0   
4                anti                  NaN                 20.0   
5             fitness                  NaN                 20.0   
6                anti                  NaN                 30.0   
7             fitness                  NaN                 20.0   
8                anti                  NaN                  NaN   
9             fitness                  NaN                  NaN   

           Method                            Bibliographic reference  
0        Deletion  2504: Smith et al. (2008) "Quantitative eviden...  
1        Deletion  2501: Matecic et al. (2010) "A microarray-base...  
2  Overexpression  2511: Easlon et al. (2008) "The malate-asparta...  
3  Overexpression  2511: Easlon et al. (2008) "The malate-asparta...  
4        Deletion  2503: Managbanag et al. (2008) "Shortest-path ...  
5        Deletion  2503: Managbanag et al. (2008) "Shortest-path ...  
6        Deletion  2504: Smith et al. (2008) "Quantitative eviden...  
7        Deletion  2504: Smith et al. (2008) "Quantitative eviden...  
8        Deletion  2500: Fabrizio et al. (2010) "Genome-wide scre...  
9        Deletion  3522: Eisenberg et al. (2014) "Nucleocytosolic...

Get the relevant values for our Network

In [12]:
longevity_id = [x for x in df_yeast['Ensembl ID']]
longevity_effect = [x for x in df_yeast['Longevity Influence']]
longevity_id_dict = dict(zip(longevity_id, longevity_effect))

In [47]:
#Make graph with color coded nodes 

colors = ['#d61111','#11d646','#11c6d6','#d67711','#1b11d6','#d611cc']

def set_color(n):
    if n in longevity_id_dict:
        if longevity_id_dict[n] == 'pro':
            # green
            return colors[1]
        elif longevity_id_dict[n] == 'anti':
            # red
            return colors[0]
        elif longevity_id_dict[n] == 'fitness':
            # orange
            return colors[3]
        elif longevity_id_dict[n] == 'unclear':
            return colors[2]
        else:
            return colors[2]
    else:
        return colors[2]
    
interactions = 'ppi/ppi.txt'
gene_dict = {}
id_int = 0
G = nx.Graph()
with open(interactions, 'r') as f:
    for line in f:
        i = line.split('\t')
        if i[0].strip() not in gene_dict:
            gene_dict[i[0].strip()] = id_int
            node_color = set_color(i[0].strip())
            G.add_node(id_int, name=i[0].strip(), color=node_color)
            id_int +=1
        if i[1].strip() not in gene_dict:
            gene_dict[i[1].strip()] = id_int
            node_color = set_color(i[0].strip())
            G.add_node(id_int, name=i[1].strip(), color = node_color)
            id_int +=1
        G.add_edge(gene_dict[i[0].strip()], gene_dict[i[1].strip()])

{'name': 'YLR418C', 'color': '#11c6d6'}


In [48]:
connected = [c for c in sorted(nx.connected_components(G), key=len, reverse=True)][0]
G_main = G.subgraph(connected).copy()

In [50]:
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
ax.set_title('Protein Interaction Network')
nx.draw(G_main, pos=nx.spring_layout(G_main),
        node_size=50, 
        node_color=[G_main.nodes[n]['color'] for n in G_main.nodes()], ax=ax)
plt.show()

**Network Facts**

In [ ]:
n = G_main.number_of_nodes()
m = G_main.number_of_edges()
k = (2*m)/n
C = nx.transitivity(G_main)

# from PS 3 
def compute_MGD(G):
    # input : a networkx graph G
    # output: the mean geodesic path length (defined in Lecture 2)
    
    # apsp is the matrix of all pairs shortest paths. It will not include infite paths. It will include 0 paths.
    apsp = dict(nx.all_pairs_shortest_path(G))
    
    # z is the number of non-infinite and non-zero pairwise distances being summed over. 
    z = 0
    # the sum of all path lengths which are non-zero and non-infinite
    sum_dist = 0
    
    for source in apsp:
        for target in apsp[source]:
            # This will remove and paths with len 0 from the apsp dict.
            if target != source:
                # using the -1 will give accurate path lengths rather than node count.
                sum_dist += len(apsp[source][target]) -1
                z += 1
    # return the mean of the shortest pairwise distances
    return sum_dist/z

ellmean = compute_MGD(G_main)


print("Overall")
print('Number of nodes: {}'.format(n))
print('Number of edges: {}'.format(m))
print('Average Degree: {}'.format(k))
print('Clustering coefficient: {}'.format(C))
print('Mean Geodesic Distance: {}'.format(ellmean))

In [ ]:
n_L = len()

In [19]:
worm_file = 'worm_genage_models_export.tsv'
df_worm = pd.read_csv(worm_file, sep=r'\t', engine='python')
df_worm.head(20)

Gene ID  Entrez Gene ID Gene Symbol  \
0         1        181727.0       aak-2   
1         1        181727.0       aak-2   
2         1        181727.0       aak-2   
3         1        181727.0       aak-2   
4         1        181727.0       aak-2   
5         1        181727.0       aak-2   
6         1        181727.0       aak-2   
7       269        181492.0      aakb-1   
8       752        176552.0      aakb-2   
9      1849        181736.0      aakg-2   
10        2        185079.0       aat-8   
11      657        183860.0      abcx-1   
12        3        173404.0      abu-11   
13      300        177654.0     acdh-13   
14      884        181706.0       ace-1   
15        4        185044.0      acl-11   
16     2101        180780.0       acn-1   
17        5        176121.0       aco-2   
18        5        176121.0       aco-2   
19        5        176121.0       aco-2   

                                            Gene Name                Organism  \
0                                AMP-Activated Kinase  Caenorhabditis elegans   
1                                AMP-Activated Kinase  Caenorhabditis elegans   
2                                AMP-Activated Kinase  Caenorhabditis elegans   
3                                AMP-Activated Kinase  Caenorhabditis elegans   
4                                AMP-Activated Kinase  Caenorhabditis elegans   
5                                AMP-Activated Kinase  Caenorhabditis elegans   
6                                AMP-Activated Kinase  Caenorhabditis elegans   
7                   AMP-Activated Kinase Beta subunit  Caenorhabditis elegans   
8                   AMP-Activated Kinase Beta subunit  Caenorhabditis elegans   
9                                      Protein AAKG-2  Caenorhabditis elegans   
10                             Amino Acid Transporter  Caenorhabditis elegans   
11                          ABC transporter, eXtended  Caenorhabditis elegans   
12     Activated in Blocked Unfolded protein response  Caenorhabditis elegans   
13                             Acyl CoA DeHydrogenase  Caenorhabditis elegans   
14                      abnormal ACEtylcholinesterase  Caenorhabditis elegans   
15                               ACyLtransferase-like  Caenorhabditis elegans   
16  Inactive angiotensin-converting enzyme-related...  Caenorhabditis elegans   
17                                          ACOnitase  Caenorhabditis elegans   
18                                          ACOnitase  Caenorhabditis elegans   
19                                          ACOnitase  Caenorhabditis elegans   

   Unigene ID Ensembl ID UniProt ID  Alias Lifespan Effect  \
0   Cel.17479    T01C8.1        NaN    NaN        decrease   
1   Cel.17479    T01C8.1        NaN    NaN        decrease   
2   Cel.17479    T01C8.1        NaN    NaN       increases   
3   Cel.17479    T01C8.1        NaN    NaN        increase   
4   Cel.17479    T01C8.1        NaN    NaN       increases   
5   Cel.17479    T01C8.1        NaN    NaN       increases   
6   Cel.17479    T01C8.1        NaN    NaN       increases   
7     Cel.860    F55F3.1        NaN    NaN        decrease   
8    Cel.9773  Y47D3A.15        NaN    NaN        decrease   
9         NaN        NaN        NaN    NaN        increase   
10  Cel.13054    F28F9.4        NaN    NaN        increase   
11  Cel.24987    C56E6.1        NaN    NaN        increase   
12  Cel.14499    T01D1.6        NaN    NaN        increase   
13   Cel.7888   C28C12.9        NaN    NaN        increase   
14        NaN   W09B12.1        NaN    NaN        increase   
15  Cel.24997    F28B3.9        NaN    NaN        increase   
16        NaN        NaN        NaN    NaN        increase   
17  Cel.17858   F54H12.1        NaN    NaN        increase   
18  Cel.17858   F54H12.1        NaN    NaN        increase   
19  Cel.17858   F54H12.1        NaN    NaN        increase   

                                Phenotype Description Longevity Influence  \
0   Could be a sen

In [53]:
mus_file = 'mus_genage_models_export.tsv'
df_mus = pd.read_csv(mus_file, sep=r'\t', engine='python')
df_mus.head(20)

Gene ID  Entrez Gene ID Gene Symbol  \
0         6          224129       Adcy5   
1      1953           11549      Adra1a   
2      1954           11548      Adra1b   
3       811           11607      Agtr1a   
4      1810           11651        Akt1   
5      2231           11652        Akt2   
6       812           11816        Apoe   
7        13          228359     Arhgap1   
8        15           11865       Arntl   
9      1862           11793        Atg5   
10       24           11920         Atm   
11       24           11920         Atm   
12       24           11920         Atm   
13       24           11920         Atm   
14       30          245000         Atr   
15       30          245000         Atr   
16       40           12028         Bax   
17       48           12189       Brca1   
18       48           12189       Brca1   
19       50           12236       Bub1b   

                                            Gene Name      Organism  \
0                                 adenylate cyclase 5  Mus musculus   
1                       adrenergic receptor, alpha 1a  Mus musculus   
2                       adrenergic receptor, alpha 1b  Mus musculus   
3                    Angiotensin II receptor, type 1a  Mus musculus   
4                      thymoma viral proto-oncogene 1  Mus musculus   
5                      thymoma viral proto-oncogene 2  Mus musculus   
6                                    Apolipoprotein E  Mus musculus   
7                     Rho GTPase activating protein 1  Mus musculus   
8   aryl hydrocarbon receptor nuclear translocator...  Mus musculus   
9                                 autophagy related 5  Mus musculus   
10      Ataxia telangiectasia mutated homolog (human)  Mus musculus   
11      Ataxia telangiectasia mutated homolog (human)  Mus musculus   
12      Ataxia telangiectasia mutated homolog (human)  Mus musculus   
13      Ataxia telangiectasia mutated homolog (human)  Mus musculus   
14             Ataxia telangiectasia and Rad3 related  Mus musculus   
15             Ataxia telangiectasia and Rad3 related  Mus musculus   
16                          Bcl2-associated X protein  Mus musculus   
17                                    Breast cancer 1  Mus musculus   
18                                    Breast cancer 1  Mus musculus   
19  budding uninhibited by benzimidazoles 1 homolo...  Mus musculus   

   Unigene ID          Ensembl ID UniProt ID  \
0    Mm.41137  ENSMUSG00000022840     P84309   
1         NaN                 NaN        NaN   
2         NaN                 NaN        NaN   
3         NaN  ENSMUSG00000049115        NaN   
4         NaN                 NaN        NaN   
5         NaN                 NaN        NaN   
6         NaN  ENSMUSG00000002985        NaN   
7   Mm.463444  ENSMUSG00000027247     Q5FWK3   
8   Mm.440371  ENSMUSG00000055116     Q9WTL8   
9         NaN                 NaN        NaN   
10    Mm.5088  ENSMUSG00000034218     Q62388   
11    Mm.5088  ENSMUSG00000034218     Q62388   
12    Mm.5088  ENSMUSG00000034218     Q62388   
13    Mm.5088  ENSMUSG00000034218     Q62388   
14  Mm.212462  ENSMUSG00000032409     Q9JKK8   
15  Mm.212462  ENSMUSG00000032409     Q9JKK8   
16   Mm.19904  ENSMUSG00000003873     Q07813   
17  Mm.244975  ENSMUSG00000017146     P48754   
18  Mm.244975  ENSMUSG00000017146     P48754   
19   Mm.29133  ENSMUSG00000040084     Q9Z1S0   

                                                Alias Lifespan Effect  \
0                                        Ac5 AW121902        increase   
1                                                 NaN        increase   
2                                                 NaN        decrease   
3   AT1 AG2S AT1a AT2R1 Agtr1 AT2R1A Agtr-1a AI551...        increase   
4                        Akt PKB Rac PKB/Akt PKBalpha        increase   
5                                                 NaN        increase   
6                                            AI255918        decrease   
7   C76222 Cdc4